# Time series forecasting

This notebook contains a brief demonstration of neural networks for time series forecasting. We use a **recurrent neural network** (RNN), in particular a **long short-term memory** (LSTM) model, and a **temporal convolutional network** (TCN) architecture in order to predict the future values of a two-variate sine/cosine time series. The forecasting models are trained with noisy data which is artificially generated for that purpose.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from tsutils import (
    SlidingWindows,
    LSTM,
    CausalConv1d,
    TCN,
    train
)

In [ ]:
np.random.seed(12345)
_ = torch.manual_seed(54321)

## Data generation

Let us start by generating a sequence of data. A sine and a cosine function are sampled at regular locations and corrupted with random noise, respectively. Those two curves constitute a bi-variate time series.

In [ ]:
num_steps = 2001
noise_level = 0.1

x_values = np.linspace(0, 100, num_steps)
noisy_sine = np.sin(x_values) + np.random.normal(loc=0, scale=noise_level, size=num_steps)
noisy_cosine = np.cos(x_values) + np.random.normal(loc=0, scale=noise_level, size=num_steps)
data = np.column_stack((noisy_sine, noisy_cosine)).astype('float32')

print('Data shape:', data.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(data[:,0], alpha=0.7, label='sine data')
ax.plot(data[:,1], alpha=0.7, label='cosine data')
ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, len(data)-1))
ax.legend(loc='lower left')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

The data is split into a training and a validation set. It is noted that due to the sequential character, this amounts to a simple division of the time series into two parts, without shuffling of any kind. This is easily done by setting the flag `shuffle=False` in the code below.

In [ ]:
val_size = 0.2

train_data, val_data = train_test_split(
    data,
    test_size=val_size,
    shuffle=False
)

print('Train data shape:', train_data.shape)
print('Val. data shape:', val_data.shape)

For training purposes, the data is best accessed through a PyTorch dataset. The `SlidingWindows` class implements a sliding windows mechanism over the full sequence. It yields overlapping intervals of data.

## LSTM

A simple LSTM neural network is now trained on the synthetically generated time series data above. After training the model, the performance is tested and compared against a naive baseline approach.

### Data set/loader

As a first step, we initialize sliding windows datasets for the training and validation sequences. Intervals of twenty consecutive time steps are considered.

In [ ]:
window_size = 20

train_set = SlidingWindows(train_data, window_size)
val_set = SlidingWindows(val_data, window_size)

print('No. train points:', len(train_set))
print('No. val. points:', len(val_set))

The corresponding data loaders are created in the following cell. Sixteen randomly sampled time intervals constitute a mini-batch.

In [ ]:
batch_size = 16

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

print('No. train batches:', len(train_loader))
print('No. val. batches:', len(val_loader))

A batch of data is loaded in order to ensure that everything works fine so far.

In [ ]:
X_batch, y_batch = next(iter(train_loader))
print('Input shape:', X_batch.shape)
print('Target shape:', y_batch.shape)

### Model training

A simple LSTM architecture is defined and initialized next. Moreover, an MSE loss function and an optimizer are set up.

In [ ]:
lstm = LSTM(
    input_size=2,
    hidden_size=5,
    num_layers=1
)

print('No. weights:', lstm.num_trainable)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)

Subsequently, the model is trained for a small number of epochs. It can be seen that the training converges fairly quickly.

In [ ]:
train(
    lstm,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    train_loader=train_loader,
    val_loader=val_loader
)

The final model outperforms the naive forecast in terms of the validation MSE.

In [ ]:
naive_mse = np.mean(np.diff(val_data, axis=0)**2)
print('Naive forecast MSE: {:.4e}'.format(naive_mse))

### Test predictions

Finally, we visualize recursively computed model forecasts starting from the first validation set window.

In [ ]:
seq = torch.as_tensor(val_data[:window_size]) # (time, features)
seq = seq.unsqueeze(0) # (batch=1, time, features)

lstm.eval()
with torch.no_grad():
    preds = lstm.forecast(
        seq,
        steps=len(val_data) - window_size
    )

print('Pred. shape:', preds.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(
    np.arange(window_size), val_data[:window_size,0],
    color=plt.cm.Dark2(0), alpha=0.7,
    label='test sine'
)
ax.plot(
    np.arange(window_size), val_data[:window_size,1],
    color=plt.cm.Dark2(1), alpha=0.7,
    label='test cosine'
)
ax.plot(
    np.arange(window_size, len(val_data)), preds[0,:,0],
    color=plt.cm.Dark2(0), alpha=0.7, linestyle='--',
    label='sine forecast'
)
ax.plot(
    np.arange(window_size, len(val_data)), preds[0,:,1],
    color=plt.cm.Dark2(1), alpha=0.7, linestyle='--',
    label='cosine forecast'
)
ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, len(val_data)-1))
ax.legend(loc='lower left')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

## TCN

As an alternative approach, a TCN model is now trained on the synthetic data set. The performance can be compared against the LSTM.

### Data set/loader

As before, we start by initializing data sets and loaders for the training and validation set, respectively.

In [ ]:
window_size = 64

train_set = SlidingWindows(
    train_data,
    window_size,
    mode='shift',
    time_last=True
)

val_set = SlidingWindows(
    val_data,
    window_size,
    mode='shift',
    time_last=True
)

print('No. train points:', len(train_set))
print('No. val. points:', len(val_set))

In [ ]:
batch_size = 16

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

print('No. train batches:', len(train_loader))
print('No. val. batches:', len(val_loader))

In [ ]:
X_batch, y_batch = next(iter(train_loader))
print('Input shape:', X_batch.shape)
print('Target shape:', y_batch.shape)

### Model training

A simple TCN architecure based on so-called "causal" convolutions is implemented. The same loss function and optimization algorithm as before are used.

In [ ]:
model = nn.Sequential(
    CausalConv1d(2, 4, kernel_size=3),
    nn.Tanh(),
    CausalConv1d(4, 4, kernel_size=3, dilation=2),
    nn.Tanh(),
    CausalConv1d(4, 2, kernel_size=3, dilation=4)
)

tcn = TCN(model)
print('No. weights:', tcn.num_trainable)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(tcn.parameters(), lr=0.001)

In [ ]:
# loss = criterion(y_pred, y_batch) # TODO: Turn off left padding for training only on actual non-padded data

train(
    tcn,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    train_loader=train_loader,
    val_loader=val_loader
)

### Test predictions

As a last step, sequential model predictions that start from the first validation set interval are plotted.

In [ ]:
seq = torch.as_tensor(val_data[:window_size]) # (time, features)
seq = seq.T.unsqueeze(0) # (batch=1, features, time)

tcn.eval()
with torch.no_grad():
    preds = tcn.forecast(
        seq,
        steps=len(val_data) - window_size
    )

print('Pred. shape:', preds.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(
    np.arange(window_size), val_data[:window_size,0],
    color=plt.cm.Dark2(0), alpha=0.7,
    label='test sine'
)
ax.plot(
    np.arange(window_size), val_data[:window_size,1],
    color=plt.cm.Dark2(1), alpha=0.7,
    label='test cosine'
)
ax.plot(
    np.arange(window_size, len(val_data)), preds[0,0,:],
    color=plt.cm.Dark2(0), alpha=0.7, linestyle='--',
    label='sine forecast'
)
ax.plot(
    np.arange(window_size, len(val_data)), preds[0,1,:],
    color=plt.cm.Dark2(1), alpha=0.7, linestyle='--',
    label='cosine forecast'
)
ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, len(val_data)-1))
ax.legend(loc='lower left')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()